In [74]:
import litellm
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()


print("Libraries are succesfully loaded!")

Libraries are succesfully loaded!


In [75]:
MODEL_PRICE = {
    "gpt-4o-mini":{
        "input_per_1m": 0.15,
        "output_per_1m":0.60
    }
}

MODEL_NAME = "openai/gpt-4o-mini"

price_info = MODEL_PRICE[MODEL_NAME.split("/")[-1]]
PRICE_PER_INPUT_TOKEN = price_info["input_per_1m"] / 1_000_000
PRICE_PER_OUTPUT_TOKEN = price_info["output_per_1m"] / 1_000_000

print(f"Price per model: {MODEL_NAME}")
print(f"Price per input token\t${PRICE_PER_INPUT_TOKEN:.10f}") 
print(f"Price per output token\t${PRICE_PER_OUTPUT_TOKEN:.10f}")

Price per model: openai/gpt-4o-mini
Price per input token	$0.0000001500
Price per output token	$0.0000006000


In [78]:
def get_completion(prompt, model, max_tokens=200):
    print("----Geting completion from LiteLLM---")

    response = litellm.completion(
        model=model, 
        messages=[
            {
            "role": "user",
            "content": prompt
            }
        ],
        max_tokens=max_tokens
    )
    return response

def analyze_cost(response):
    if not response:
        print("Cannot analyze cost of a failed API call")
        return

    raw_model_name = response.model

    if "/" in raw_model_name:
        raw_model_name = raw_model_name.split("/")[-1]

    model_used = None

    for model in MODEL_PRICE.keys():
        if raw_model_name.startswith(model):
            model_used = model
            break

    if not price_info:
        print(f"Warning: pricing for model '{model_used}' not found!")
        return

    usage_data = response.usage
    input_tokens = usage_data.prompt_tokens
    output_tokens = usage_data.completion_tokens
    price_per_input = price_info["input_per_1m"] / 1_000_000
    price_per_output = price_info["output_per_1m"] / 1_000_000
    
    input_cost = input_tokens * price_per_input
    output_cost = output_tokens * price_per_output

    print(f"Price per model: {model_used}")
    print(f"Model reponse: {response.choices[0].message.content}")
    print(f"input token\t{input_tokens:}\t cost: ${input_cost:.8f}") 
    print(f"output token\t{output_tokens:}\t cost: ${output_cost:.8f}")

user_prompt = "what are top 5 actions an great team leder does for it's team?"

response = get_completion(
    user_prompt,
    model=MODEL_NAME
)


----Geting completion from LiteLLM---


In [81]:
analyze_cost(response)

Price per model: gpt-4o-mini
Model reponse: A great team leader implements several key actions to foster a productive and positive team environment. Here are the top five actions:

1. **Establish Clear Communication**: Effective communication is essential for teamwork. A great leader ensures that information flows freely, encourages open dialogue, and creates a culture where team members feel comfortable sharing ideas and feedback.

2. **Set Clear Goals and Expectations**: A strong leader sets well-defined, achievable goals for the team, aligning them with the overall vision of the organization. This helps to provide direction, purpose, and a sense of accountability among team members.

3. **Foster Collaboration and Trust**: A great leader promotes a collaborative environment by encouraging teamwork and building trust. This includes recognizing individual contributions, celebrating successes, and supporting team members in their development.

4. **Provide Support and Resources**: Great